# Семинар 6 - Композиции алгоритмов

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(palette='deep', style='darkgrid', rc={"figure.figsize": (15, 4)})
import scipy.stats as st

import warnings
warnings.simplefilter('ignore')

In [ ]:
# Загрузим данные и проведем все предобработки как на семинаре: 
data = pd.read_csv('../data/flight_delays_train.csv')
data['dep_delayed_15min'] = data['dep_delayed_15min'].apply(lambda x: 1 if x == 'Y' else 0)
data['Month'] = data['Month'].str.replace('c-', '').astype('int16')
data['DayofMonth'] = data['DayofMonth'].str.replace('c-', '').astype('int16')
data['DayOfWeek'] = data['DayOfWeek'].str.replace('c-', '').astype('int16')
data['UniqueCarrier'] = pd.factorize(data['UniqueCarrier'])[0]
data['Origin'] = pd.factorize(data['Origin'])[0]
data['Dest'] = pd.factorize(data['Dest'])[0]

x = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'].values

data.shape

In [ ]:
# Пусть бдет более вещественным числом (так как 60 минут в часах)
data['DepTime_real'] = data['DepTime'].apply(lambda x: int(x/100)+((x/100-int(x/100))*100)/59)

# Bootstrap
Посмотрим плотности распредления переменной "Время Вылета" для задержки менее 15 минут и более

In [ ]:
sns.kdeplot(data[data['dep_delayed_15min'] == 0]['DepTime_real'], label='Задержка рейса менее 15 мин')
sns.kdeplot(data[data['dep_delayed_15min'] == 1]['DepTime_real'], label='Задержка рейса более 15 мин')
plt.legend()
plt.show()

In [ ]:
print('Среднее', data[data['dep_delayed_15min'] == 1]['DepTime_real'].mean())
print('Среднее', data[data['dep_delayed_15min'] == 0]['DepTime_real'].mean())

In [ ]:
def get_bootstrap_samples(data, n_samples):
    # функция для генерации подвыборок с помощью бутстрэпа
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    # функция для интервальной оценки
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [ ]:
# сохранение в отдельные numpy массивы данных по вылетам с задержками и без 
no_delayed = data[data['dep_delayed_15min'] == 0]['DepTime_real'].values
delayed = data[data['dep_delayed_15min'] == 1]['DepTime_real'].values

# ставим seed для воспроизводимости результатов
np.random.seed(0)

# генерируем 1000 выборок с помощью бутстрэпа и сразу считаем по каждой из них среднее
no_delayed_mean_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(no_delayed, 1000)]
delayed_mean_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(delayed, 1000)]

In [ ]:
#  выводим интервальную оценку среднего
print("Среднее время вылета по рейсам без задержек в интервале:",  stat_intervals(no_delayed_mean_scores, 0.05))
print("Среднее время вылета по рейсам с задержками в интервале:",  stat_intervals(delayed_mean_scores, 0.05))

Sub-sampling (_pasting_) тоесть выборка без повторений - достойная альтернатива

# Bagging (Bootstrap aggregating)

In [ ]:
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
tree = DecisionTreeClassifier()
bag_of_trees = BaggingClassifier(tree, n_estimators=20)

In [ ]:
# Посчитаем значение метрики accuracy на кроссвалидаци для дерева
np.mean(cross_val_score(tree, x, y, cv=3, scoring='accuracy'))

In [ ]:
# Посчитаем значение метрики accuracy на кроссвалидаци для композиции деревьев построенной на бутстрап выборке
np.mean(cross_val_score(bag_of_trees, x, y, cv=3, scoring='accuracy'))

### Out-of-bag error

На каждом шаге все объекты попадают в подвыборку с возвращением равновероятно, значит:  
Вероятность, что объект попадет в выборку: $ \frac {1}{l}$   

Вероятность, что объект не попадет в выборку: $ 1-\frac {1}{l}$    
  
Так как мы тянем $l$ раз, то  вероятность, что объект не попадет во всю выборку: $ \bigl( 1-\frac {1}{l} \bigr) ^l$ 

Значит, при  $l \rightarrow \infty$ что вероятность, что объект не поппадает в выборку: $ \frac {1}{e} \approx 0.37 $      
  
__Вывод:__ При формировании Bootstrap выборки в нее попадает только __63%__ объектов   
__Свойство:__ Можно вычислять Out-of-bag error и не проводить кроссвалидацию

In [ ]:
tree = DecisionTreeClassifier()
bag_of_trees = BaggingClassifier(tree, n_estimators=20, oob_score=True, n_jobs=-1)

In [ ]:
bag_of_trees.fit(x,y)
bag_of_trees.oob_score_

### Как можно добавить случайности? 

Например: Ограничить кол-во признаков, по которым проводить разбиение

In [ ]:
tree = DecisionTreeClassifier(max_features=2)
bag_of_trees = BaggingClassifier(tree, n_estimators=20, oob_score=True, n_jobs=-1)

bag_of_trees.fit(x,y)
bag_of_trees.oob_score_

Почему это работает, и зачем нам нужна случайность?

# Случайный лес

In [ ]:
forest = RandomForestClassifier(n_estimators=20, n_jobs=-1)
np.mean(cross_val_score(forest, x,y, cv=3, n_jobs=-1, scoring='accuracy'))




### Что будет, если ограничить глубину построенных деревьев? 

In [ ]:
#Разделим выборку на обущающую и тестовую
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, test_size=0.3, \
                                                    shuffle=True, random_state=21)

In [ ]:
train_score = []
test_score = []


for i in range(50):
    forest = RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=i+1, min_samples_leaf=50)
    forest.fit(x_train, y_train)
    train_score = np.append(train_score, accuracy_score(y_train, forest.predict(x_train)))
    test_score = np.append(test_score, accuracy_score(y_test, forest.predict(x_test)))

In [ ]:
plt.plot(train_score)
plt.plot(test_score)
plt.show()